In [1]:
import os
import torch
import argparse
import numpy as np
import pandas as pd
from util import * 
from model import *
from pandas import DataFrame
from sklearn import metrics

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/lijiahao/anaconda3/envs/tensorflow_pytorch_python/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--epoches',              type=int,  default=30,  help='')
parser.add_argument('--batch_size',           type=int,  default=128,  help='')
parser.add_argument('--max_length',           type=int,  default=200, help='')
parser.add_argument('--learning_rate',        type=float, default=1e-4, help="")
parser.add_argument('--model_path',           type=str,  default="../3-new-12w-0", help='')
parser.add_argument('--ind_filename',  type=str,  default="../dataset/enhancer_3-mer_DNABERT_ind.txt", help='')
parser.add_argument('--tra_filename',  type=str,  default="../dataset/enhancer_3-mer_DNABERT_tra.txt", help='')

args = parser.parse_args(args=[]) # 如果不使用"args=[]"，会报错

In [3]:
def embedding(sequences, DNABert, tokenizer):
    
    output_embeddings = []

    for sequence in sequences:
        sequence_1 = " ".join(sequence.split(" ")[0:500])
        sequence_2 = " ".join(sequence.split(" ")[500:1000])
        sequence_3 = " ".join(sequence.split(" ")[1000:1500])
        sequence_4 = " ".join(sequence.split(" ")[1500:2000])

        number_1 = len(sequence_1.split(" "))
        number_2 = len(sequence_2.split(" "))
        number_3 = len(sequence_3.split(" "))
        number_4 = len(sequence_4.split(" "))

        # sequence_1 = tokenizer(sequence_1, padding='max_length', truncation=True, max_length = 502)
        # sequence_2 = tokenizer(sequence_2, padding='max_length', truncation=True, max_length = 502)
        # sequence_3 = tokenizer(sequence_3, padding='max_length', truncation=True, max_length = 502)
        # sequence_4 = tokenizer(sequence_4, padding='max_length', truncation=True, max_length = 502)

        tokening = tokenizer([sequence_1, sequence_2, sequence_3, sequence_4], padding='max_length', truncation=True, max_length = 502)
 
        batch_ids = torch.IntTensor(tokening["input_ids"]).to(device)
        batch_mask = torch.IntTensor(tokening["attention_mask"]).to(device)
        batch_number = torch.IntTensor([number_1, number_2, number_3, number_4]).to(device)

        DNABert.eval()
        with torch.no_grad():
            embedding_1 = DNABert(batch_ids, batch_mask, batch_number).to("cpu").numpy()
        output_embeddings.append(embedding_1)

    return output_embeddings


def train_classifier(model, dataloader, optimizer):
    train_iter, train_loss_sum = 0.0, 0.0
    real_labels, pre_labels = [], []
    for batch_data in dataloader:
        batch_features = batch_data["features"].to(device)
        batch_labels = batch_data["labels"].to(device)

        model.train()
        optimizer.zero_grad()
        outputs = model(batch_features, batch_labels)
        
        # 反向梯度信息
        loss = outputs[0]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 2.0)
        # 参数更新
        optimizer.step()

        # compute performance
        train_loss = outputs[0].detach().to("cpu").numpy()
        logits = outputs[1].detach()

        train_iter += 1
        train_loss_sum += train_loss

        if len(real_labels) == 0:
            real_labels = batch_labels.to("cpu").numpy()
            pre_labels = logits.to("cpu").numpy()
        else:
            real_labels = np.concatenate([real_labels, batch_labels.to("cpu").numpy()], axis=0)
            pre_labels = np.concatenate([pre_labels, logits.to("cpu").numpy()], axis=0)

    tra_loss = train_loss_sum/(train_iter)
    acc, mcc, sn, sp = evaluation_criterion(pre_labels, real_labels)

    fpr, tpr, threshold = metrics.roc_curve(real_labels, pre_labels)
    roc_auc = metrics.auc(fpr, tpr)

    return tra_loss, acc, mcc, sn, sp, roc_auc


def test_classifier(model, dataloader, optimizer):
    test_iter, test_loss_sum = 0.0, 0.0
    real_labels, pre_labels = [], []
    for batch_data in dataloader:
        batch_features = batch_data["features"].to(device)
        batch_labels = batch_data["labels"].to(device)

        model.eval()
        with torch.no_grad():
            outputs = model(batch_features, batch_labels)
        
        # compute performance
        test_loss = outputs[0].detach().to("cpu").numpy()
        logits = outputs[1].detach()

        test_iter += 1
        test_loss_sum += test_loss

        if len(real_labels) == 0:
            real_labels = batch_labels.to("cpu").numpy()
            pre_labels = logits.to("cpu").numpy()
        else:
            real_labels = np.concatenate([real_labels, batch_labels.to("cpu").numpy()], axis=0)
            pre_labels = np.concatenate([pre_labels, logits.to("cpu").numpy()], axis=0)

    test_loss = test_loss_sum/(test_iter)
    acc, mcc, sn, sp = evaluation_criterion(pre_labels, real_labels)

    fpr, tpr, threshold = metrics.roc_curve(real_labels, pre_labels)
    roc_auc = metrics.auc(fpr, tpr)

    return test_loss, acc, mcc, sn, sp, roc_auc, real_labels, pre_labels



In [4]:
# average [3,4,5,6]
# 添加L2-正则化 average
# cell_lines = ["GM12878", "HUVEC", "HSMM", "HEK293", "HMEC", "K562", "NHEK", "NHLF"]
cell_lines = ["NHEK"]

final_filename = "./result/record_NHEK.txt"
final_log = open(final_filename, "w")
content = "state\ttrain-loss\tacc\tmcc\tsn\tsp\tauc\ttest-loss\tacc\tmcc\tsn\tsp\tauc\tind-loss\tacc\tmcc\tsn\tsp\tauc\n"
final_log.write(content)
final_log.flush()

for cell_line in cell_lines:
    filename = "./result/{}.txt".format(cell_line)
    file_log = open(filename, "w")
    content = "state\ttrain-loss\tacc\tmcc\tsn\tsp\tauc\ttest-loss\tacc\tmcc\tsn\tsp\tauc\tind-loss\tacc\tmcc\tsn\tsp\tauc\n"
    file_log.write(content)
    file_log.flush()

    mers = [3, 4, 5, 6]
    test_real_labels_list, test_pre_labels_list = [], []

    for i in range(len(mers)):
        mer = mers[i]
        
        args.model_path = "../../DNA-BERT/{}-new-12w-0".format(mer)
        
        tokenizer = BertTokenizer.from_pretrained(args.model_path)
        DNABert = C_Bert_average_embedding.from_pretrained(args.model_path).to(device)

        args.ind_filename = "/home/lijiahao/project/Enhancer/Enhancer-IF/dataset/process/test-dataset(all-length)/{}-test-{}mer.txt".format(cell_line, mer)
        args.tra_filename = "/home/lijiahao/project/Enhancer/Enhancer-IF/dataset/process/train-dataset(all-length)/{}-train-{}mer.txt".format(cell_line, mer)
        
        tra_df_raw = pd.read_csv(args.tra_filename, sep="\t",header=None,names=["text","label"]) 
        ind_df_raw = pd.read_csv(args.ind_filename, sep="\t",header=None,names=["text","label"]) 
        tra_set, val_set = train_test_split(tra_df_raw, stratify=tra_df_raw['label'], test_size=0.1, random_state=42)

        tra_text, tra_labels = list(tra_set["text"]), list(tra_set["label"])
        val_text, val_labels = list(val_set["text"]), list(val_set["label"])
        ind_text, ind_labels = list(ind_df_raw["text"]), list(ind_df_raw["label"])

        tra_features = embedding(tra_text, DNABert, tokenizer)
        val_features = embedding(val_text, DNABert, tokenizer)
        ind_features = embedding(ind_text, DNABert, tokenizer)    

        print("----------Embedding end!!!--------------")
        tra_dataset = NewDataset_classifier(tra_features, tra_labels)
        tra_dataloader = DataLoader(tra_dataset, batch_size = args.batch_size, shuffle = True)

        val_dataset = NewDataset_classifier(val_features, val_labels)
        val_dataloader = DataLoader(val_dataset, batch_size = args.batch_size, shuffle = True)

        ind_dataset = NewDataset_classifier(ind_features, ind_labels)
        ind_dataloader = DataLoader(ind_dataset, batch_size = args.batch_size, shuffle = False)

        print("-----------------classifier-------------")
        classifier_model = Enhancer_classifier_128().to(device)
        epoches = args.epoches
        learning_rate = args.learning_rate
        optimizer = optim.Adam(classifier_model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08,)
        scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.98)    # 指数衰减损失函数
        
        acc_max = 0.0
        temp_logits = []
        content_temp = ""
        for epoch in range(epoches):
            tra_loss, tra_acc, tra_mcc, tra_sn, tra_sp, tra_auc = train_classifier(classifier_model, tra_dataloader, optimizer)
            val_loss, val_acc, val_mcc, val_sn, val_sp, val_auc, _, _ = test_classifier(classifier_model, val_dataloader, optimizer)
            ind_loss, ind_acc, ind_mcc, ind_sn, ind_sp, ind_auc, real_labels, pre_labels = test_classifier(classifier_model, ind_dataloader, optimizer)
            content = "epoch={}-{}mer-\t{:.5f}\t{:.5f}\t{:.5f}\t{:.5f}\t{:.5f}\t{:.5f}\t{:.5f}\t{:.5f}\t{:.5f}\t{:.5f}\t{:.5f}\t{:.5f}\t{:.5f}\t{:.5f}\t{:.5f}\t{:.5f}\t{:.5f}\t{:.5f}\n".format(epoch, mer,\
                tra_loss, tra_acc, tra_mcc, tra_sn, tra_sp, tra_auc, val_loss, val_acc, val_mcc, val_sn, val_sp, val_auc, ind_loss, ind_acc, ind_mcc, ind_sn, ind_sp, ind_auc)
            
            file_log.write(content)
            file_log.flush()

            print(content[:-1])

            if np.abs(val_sn-val_sp) < 0.15 and acc_max < val_acc:
                acc_max = val_acc
                temp_logits = pre_labels
                content_temp = content

        test_pre_labels_list.append(temp_logits)
        test_real_labels_list.append(real_labels)

        final_log.write("{}-".format(cell_line) + content_temp)
        final_log.flush()

    test_real_labels = np.mean(test_real_labels_list, axis=0)
    test_pre_labels = np.mean(test_pre_labels_list, axis=0)

    acc, mcc, sn, sp = evaluation_criterion(test_pre_labels, test_real_labels)
    fpr, tpr, threshold = metrics.roc_curve(test_real_labels, test_pre_labels)
    roc_auc = metrics.auc(fpr, tpr)

    content = "integrate-{}-ind_acc={:.5f}; ind_mcc={:.5f}; ind_sn={:.5f}; ind_sp={:.5f}; ind_AUC={:.5f}\n".format(cell_line, acc, mcc, sn, sp, roc_auc)
    print(content, end="")
    final_log.write(content)
    final_log.flush()


Some weights of the model checkpoint at ../../3-new-12w-0 were not used when initializing C_Bert_average_embedding: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing C_Bert_average_embedding from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing C_Bert_average_embedding from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


----------Embedding end!!!--------------
-----------------classifier-------------
epoch=0-3mer-	0.67491	0.55698	0.12614	0.34267	0.77130	0.60245	0.64412	0.56379	0.19233	0.18966	0.93793	0.71300	0.60727	0.69361	0.21372	0.19436	0.94323	0.73337
epoch=1-3mer-	0.62708	0.64256	0.29718	0.50153	0.78358	0.71962	0.61150	0.65690	0.32307	0.53793	0.77586	0.74583	0.59442	0.71520	0.35982	0.57473	0.78544	0.76326
epoch=2-3mer-	0.60618	0.67287	0.34911	0.60361	0.74213	0.73996	0.60260	0.70690	0.41864	0.78276	0.63103	0.75819	0.62463	0.67672	0.38727	0.79089	0.61963	0.77462
epoch=3-3mer-	0.59278	0.67479	0.35134	0.62471	0.72487	0.75091	0.58350	0.70172	0.40577	0.64828	0.75517	0.76295	0.56756	0.71969	0.39433	0.65427	0.75241	0.78108
epoch=4-3mer-	0.58497	0.68611	0.37442	0.63200	0.74021	0.75749	0.59615	0.70690	0.42717	0.83103	0.58276	0.76661	0.63108	0.66945	0.40478	0.84413	0.58210	0.78519
epoch=5-3mer-	0.59044	0.67114	0.34300	0.63891	0.70338	0.74690	0.57665	0.71379	0.42824	0.74138	0.68621	0.76723	0.58070	0.69852	0.

Some weights of the model checkpoint at ../../4-new-12w-0 were not used when initializing C_Bert_average_embedding: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing C_Bert_average_embedding from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing C_Bert_average_embedding from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


----------Embedding end!!!--------------
-----------------classifier-------------
epoch=0-4mer-	0.66680	0.57982	0.17616	0.36838	0.79125	0.63493	0.64348	0.60862	0.22489	0.47931	0.73793	0.68747	0.62935	0.68591	0.29318	0.52854	0.76459	0.72637
epoch=1-4mer-	0.63327	0.62682	0.25835	0.53185	0.72180	0.69644	0.61892	0.65000	0.30998	0.52414	0.77586	0.72199	0.59050	0.71670	0.35387	0.54715	0.80148	0.75954
epoch=2-4mer-	0.61052	0.66443	0.33239	0.59171	0.73715	0.73161	0.60896	0.68448	0.37138	0.62759	0.74138	0.73389	0.58853	0.70301	0.36630	0.65170	0.72867	0.76903
epoch=3-4mer-	0.59873	0.67978	0.36113	0.63315	0.72640	0.74654	0.60442	0.67759	0.35528	0.66552	0.68966	0.73832	0.59141	0.69403	0.37915	0.71841	0.68185	0.77320
epoch=4-4mer-	0.59500	0.67441	0.35057	0.62433	0.72448	0.74433	0.60194	0.67414	0.34863	0.65172	0.69655	0.74224	0.57897	0.69852	0.37766	0.69852	0.69852	0.77697
epoch=5-4mer-	0.58514	0.68438	0.36947	0.65349	0.71527	0.75667	0.59900	0.67069	0.34678	0.58276	0.75862	0.74816	0.55802	0.71777	0.

Some weights of the model checkpoint at ../../5-new-12w-0 were not used when initializing C_Bert_average_embedding: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing C_Bert_average_embedding from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing C_Bert_average_embedding from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


----------Embedding end!!!--------------
-----------------classifier-------------
epoch=0-5mer-	0.67537	0.55219	0.12263	0.28972	0.81466	0.60304	0.66023	0.62931	0.25930	0.66552	0.59310	0.67685	0.68119	0.64058	0.27632	0.66325	0.62925	0.69623
epoch=1-5mer-	0.64869	0.61090	0.23245	0.46124	0.76055	0.67592	0.63950	0.61552	0.24932	0.42759	0.80345	0.69973	0.61916	0.68890	0.26703	0.43489	0.81591	0.71900
epoch=2-5mer-	0.63802	0.62183	0.25160	0.49731	0.74635	0.68705	0.64175	0.64828	0.30983	0.79310	0.50345	0.71624	0.68086	0.61492	0.32389	0.82745	0.50866	0.73307
epoch=3-5mer-	0.62765	0.64256	0.28727	0.58135	0.70376	0.70500	0.63201	0.60172	0.23492	0.35172	0.85172	0.72401	0.57777	0.70130	0.27483	0.37588	0.86402	0.74044
epoch=4-5mer-	0.61744	0.65004	0.30379	0.57214	0.72794	0.71999	0.61882	0.64828	0.31768	0.46897	0.82759	0.73068	0.57527	0.70772	0.31064	0.45799	0.83258	0.74763
epoch=5-5mer-	0.61279	0.65196	0.30720	0.57905	0.72487	0.72281	0.61028	0.66897	0.33952	0.62069	0.71724	0.73765	0.59841	0.69211	0.

Some weights of the model checkpoint at ../../6-new-12w-0 were not used when initializing C_Bert_average_embedding: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing C_Bert_average_embedding from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing C_Bert_average_embedding from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


----------Embedding end!!!--------------
-----------------classifier-------------
epoch=0-6mer-	0.68947	0.53089	0.07306	0.26401	0.79777	0.55463	0.66801	0.60862	0.22578	0.47241	0.74483	0.65509	0.68233	0.65683	0.22182	0.46953	0.75048	0.68284
epoch=1-6mer-	0.66011	0.59210	0.20449	0.37490	0.80929	0.65685	0.65456	0.56724	0.16977	0.26207	0.87241	0.68502	0.62893	0.69040	0.22303	0.27582	0.89769	0.71555
epoch=2-6mer-	0.64925	0.60035	0.21440	0.42441	0.77629	0.67408	0.64269	0.62069	0.24476	0.53793	0.70345	0.69486	0.63724	0.68334	0.31235	0.59269	0.72867	0.72717
epoch=3-6mer-	0.64145	0.61972	0.24657	0.50038	0.73906	0.68409	0.62854	0.61897	0.25314	0.44828	0.78966	0.70812	0.60671	0.70344	0.31486	0.49968	0.80532	0.73748
epoch=4-6mer-	0.62499	0.64524	0.29772	0.53569	0.75480	0.71383	0.62516	0.66552	0.33607	0.75172	0.57931	0.71788	0.64976	0.65277	0.34116	0.76459	0.59686	0.74409
epoch=5-6mer-	0.62300	0.64390	0.29207	0.55871	0.72909	0.71416	0.66008	0.64655	0.35240	0.92414	0.36897	0.72189	0.75001	0.56468	0.